<a href="https://colab.research.google.com/github/tara3165/NLP-/blob/CompliantAnalysis/ComplaintAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
/Colab Notebooks/Complaint

drive  sample_data


In [14]:
cd drive

/content/drive


In [18]:
import pandas as pd
complaints=pd.read_csv('../drive/MyDrive/Colab Notebooks/Complaint/complaints.csv',low_memory=False)

In [27]:
complaints=complaints[~complaints['Consumer complaint narrative'].isna()]
Description_to_process='Consumer complaint narrative'

1685238

In [33]:


from nltk.corpus import stopwords
import string
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
import spacy
#get_ipython().magic(u'matplotlib inline')

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


import os
import sys
import json
import pandas as pd
import numpy as np
import re
import time

class TopicModeling(object):
    def __init__(self,datasets, Description,penality, domain_stopwords):
        self.datasets=datasets

        self.penality=penality
        self.sentences=datasets[Description]
        self.prepocessed_words=[]

        self.number_topics=None
        self.avg_cosine_similarity=None
        self.coherence_score=None
        self.perplexity=None
        self.Best_topic_numbers=None
        self.bigram_mod=None
        self.trigram_mod=None
        self.data_words_bigrams=None
        self.data_words_trigrams=None
        self.data_lemmatized=None
        self.id2word=None
        self.corpus=None
        self.LDA_model=None
        self.topics_number=None
        self.labels=None
        self.plot=None

        self.domain_stopwords=domain_stopwords

    #run prepocessing part, and then run the LDA_Modeling part
    def Run_Prepocessing(self):
        self.text_process()
        self.build_gram_model()
        self.make_bigrams()
        self.make_trigrams()
        self.lemmatization()
        self.build_Id2Word_Corpus()


    #LDA topic modeling part includeing getAvgCosineSimilarity method and getTopicTenKeyWord method
    def Run_LDA_Model(self,number_topics):
        self.number_topics=number_topics
        self.build_LDA_model()
        # self.visualize_LDA_model()
        self.getAvgCosineSimilarity()
        self.getTopicTenKeyWord()
        self.updateDataset()



    #select the number of topics based on average cosine similarity, the smaller of average cosine similarity, the better the results because it can
    #seperate each topics very well.
    def select_Best_Number_Topics(self):
        best=1
        best_number_topics=0
        for i in range(0,len(self.topics_list)):
            number_topics=self.topics_list[i]
            avg_cosine_similarity=self.avg_cosine_similarity_list[i]
            if avg_cosine_similarity<best:
                best=avg_cosine_similarity
                best_number_topics=number_topics
        self.Best_topic_numbers=best_number_topics


    #make the plot of average cosine similarity.
    def MakePlot(self):
        X=self.topics_list
        Y=self.avg_cosine_similarity_list

        plt.plot(X,Y)
        plt.xlabel("The number of clusters")
        plt.ylabel("The Avg cosine similarity score")
        plt.label("The number of culsters with the Avg cosine similarity score list")
        plt.show()


    #print out the topic word
    def printOutTopicWords(self):
        result=TopicModeling(self.dataset,self.Best_topic_numbers,self.penality)
        result.Run_LDA_Model()   #result is the class
        final_model=result.LDA_model
        plot=result.visualize_LDA_model()
        #return plot



    #text process including transfer to lower case, remove punctuation, replace http to url.
    def text_process(self):
        for mess in self.sentences:
            partial_clean_word=[word.lower() for word in mess.split() if word not in stopwords.words("english")] #remove stop words
            remove_At_word=self.Words_Removed_At(partial_clean_word) #remove @people words
            new_sentence=" ".join(remove_At_word)

            #remove the punctuation
            remove_punc=[char for char in new_sentence if char not in string.punctuation] #remove punctuation
            remove_punc="".join(remove_punc)
            res_list=remove_punc.split(" ")
            res_list=[word for word in res_list if word!="" and word.startswith("http")==False] #remove http
            self.prepocessed_words.append(res_list)

    def Words_Removed_At(self,words): #remove @ people and sentences
        alist=[]
        for i in range(0,len(words)):
            word=words[i]
            if word.startswith("@")==False:
                alist.append(word)
        return alist


    #build the gram model
    def build_gram_model(self):
        bigram = gensim.models.Phrases(self.prepocessed_words, min_count=5, threshold=100) # higher threshold fewer phrases.
        trigram = gensim.models.Phrases(bigram[self.prepocessed_words], threshold=100)

        # Faster way to get a sentence clubbed as a trigram/bigram
        bigram_mod = gensim.models.phrases.Phraser(bigram)
        trigram_mod = gensim.models.phrases.Phraser(trigram)

        self.bigram_mod=bigram_mod
        self.trigram_mod=trigram_mod

    def make_bigrams(self):
        self.data_words_bigrams=[self.bigram_mod[doc] for doc in self.prepocessed_words]

    def make_trigrams(self):
        self.data_words_trigrams=[self.trigram_mod[self.bigram_mod[doc]] for doc in self.prepocessed_words]


    #words lemmatization. only allow the non, adj, verb, adv words. it will remove other types of words in a sentence.
    def lemmatization(self,allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        """https://spacy.io/api/annotation"""
        nlp = spacy.load('en', disable=['parser', 'ner'])
        texts_out = []
        for sent in self.data_words_bigrams:
            doc = nlp(" ".join(sent))
            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        self.data_lemmatized=texts_out


    #build the id2word corpus
    def build_Id2Word_Corpus(self):
        self.id2word=corpora.Dictionary(self.data_lemmatized)
        self.corpus=[self.id2word.doc2bow(text) for text in self.data_lemmatized]


    #build the LDA topic modeling by using some empirical parameters.
    def build_LDA_model(self):
        self.LDA_model=gensim.models.ldamodel.LdaModel(corpus=self.corpus,
                                           id2word=self.id2word,
                                           num_topics=self.number_topics,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto",
                                           per_word_topics=True)

    def getLDA_model(self):
        return self.LDA_model


    #visualize the LDA model by the power of pyLDAvis package
    def visualize_LDA_model(self):
        #pyLDAvis.enable_notebook()
        vis=pyLDAvis.gensim.prepare(self.LDA_model,self.corpus,self.id2word)
        self.plot=vis

        #return vis

    def getId2Word(self):
        return self.id2word

    def getCorpus(self):
        return self.corpus



    #get the average cosine similarity
    def getAvgCosineSimilarity(self):
        Sum=0
        topics=self.LDA_model.print_topics()
        topics_number=len(topics)
        for j in range(0,len(topics)-1):
            sentence_1=self.getTopicScoreList(topics[j][1])
            for k in range(j+1,len(topics)):
                sentence_2=self.getTopicScoreList(topics[k][1])
                Sum+=self.getCosineSimilarity(sentence_1,sentence_2)
        avg_cosine=Sum/(topics_number*(topics_number-1)/2)
        avg_cosine+=(self.penality*topics_number)
        return avg_cosine

    #get the perplexity of current model
    def getPerplexity(self):
        perplexity=self.LDA_model.log_perplexity(self.corpus)
        return perplexity

    #get the coherence score of the current model
    def getCoherence(self):
        coherence_model_lda = CoherenceModel(model=self.LDA_model,texts=self.data_lemmatized, dictionary=self.id2word, coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
        return coherence_lda



    #helper function to get Cosine similarity
    def getTopicScoreList(self,sentence):
        adict=dict()
        for atuple in sentence.split("+"):
            tuple_list=[]
            word=atuple.split("*")[1]
            number=float(atuple.split("*")[0])
            adict[word]=number
        return adict


    #get the cosine similarity of two sentences
    def getCosineSimilarity(self,sentence1,sentence2):
        Sum=0
        for key1 in sentence1:
            if key1 in sentence2:
                Sum+=sentence1[key1]*sentence2[key1]

        Sum_1=0
        for key1 in sentence1:
            Sum_1+=sentence1[key1]**2
        Sum_1=np.sqrt(Sum_1)

        Sum_2=0
        for key2 in sentence2:
            Sum_2+=sentence2[key2]**2
        Sum_2=np.sqrt(Sum_2)

        return Sum/(Sum_1*Sum_2)

    #get the top 10 key word
    def getTopicTenKeyWord(self):
        sentences=self.LDA_model.print_topics()
        all_topic=[]
        for i in range(0,len(sentences)):
            onesentence=sentences[i][1] #all string
            number=0
            one_topic=[]
            for j in range(0,len(onesentence.split("+"))):
                number_word=onesentence.split("+")[j]   #the format is like "0.082*"nepal" '
                mess_word=number_word.split("*")[1]
                clean_word=self.getCleanWord(mess_word)
                if number<10:
                    one_topic.append(clean_word)
                    number+=1
            one_topic=" ".join(one_topic)
            all_topic.append(one_topic)

        #then we will print out every key word in the topic list

        self.topic_names=all_topic

    def getEachTopicNames(self):
        for m in range(0,len(self.topic_names)):
            print("The "+str(m+1)+" Topics Meanings are: "+str(self.topic_names[m]))


    def getCleanWord(self,aword):
        new_word=""
        for letter in aword:
            if letter!="'" and letter!='"' and letter!=" ":
                new_word+=letter
        return new_word
        new_word=new_word.strip()



    def getPercentageAndTopics(self):
        percent_Topic =self.LDA_model[self.corpus]
        percent_list=[]
        topic_list=[]

        for i in range(0,len(percent_Topic)):
            res=percent_Topic[i][0]

            highest_percent=0
            best_topic=None
            for j in range(0,len(res)):
                if res[j][1]>highest_percent:
                    highest_percent=res[j][1]
                    best_topic=self.topic_names[j]

            percent_list.append(highest_percent)
            topic_list.append(best_topic)
        return percent_list,topic_list


    def updateDataset(self):
        percent_list,topic_list=self.getPercentageAndTopics()
        self.datasets["Topic_Percent"]=percent_list
        self.datasets["Topic"]=topic_list

